In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
date_envio = '2024-01-31'
monto_me_mail = 416929.49
comision_bnte_mail = 823.50
iva_bnte_mail = 131.76
monto_fondear_mail = 417884.75
date_envio = pd.to_datetime(date_envio)

# READING FILES

## BNTE

In [3]:
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                  ,dtype={'BIN':object,'Clave de la institución':object,'Nombre de la institución':object})

In [4]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

In [5]:
query_1 = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\BNTE-3 20*.csv', recursive=False)
query_1

['G:\\.shortcut-targets-by-id\\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\\3.-Settlement\\5.-HISTORICO FILES PAGOS\\01. TRADICIONAL\\2024\\01. JANUARY\\BNTE-3 2024-01-31.csv']

In [6]:
query = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                   .assign(file_name=os.path.basename(f)) for f in query_1])
query.columns = query.columns.str.strip().str.lower() 
query['CLABE'] = query['bank_account_number'].str[1:]
query['BIN'] = query['bank_account_number'].str[1:4].replace(' ', '',regex=True)
query['date_disbursement'] = date_envio

In [7]:
query_final = query.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
query_final.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,...,bank_account_number,batch_created_date,disbursement_method,swap,file_name,CLABE,BIN,date_disbursement,Clave de la institución,Nombre de la institución
0,NaN,00bf0fb9-405a-4b41-9ace-14dfe8e0b471,NaN,NaN,NaN,AHN3VLNX6,2024-01-31,324.85,PAID,AZTECA,...,'127456013890994028,2024-01-31T14:00:16Z,BANORTE,"Omar Polo Ochoa,,324.85,127456013890994028,CLI...",BNTE-3 2024-01-31.csv,127456013890994028,127,2024-01-31,40127,AZTECA
1,NaN,0194010b-f10f-48fd-9b19-5fb77fea13ea,NaN,NaN,NaN,ASSKDM3KS,2024-01-31,711.39,PAID,"Opciones Empresariales del Noreste, S.A. de C....",...,'659802020000196080,2024-01-31T14:00:16Z,BANORTE,"Alan Orlando Gonzalez Rodriguez,,711.39,659802...",BNTE-3 2024-01-31.csv,659802020000196080,659,2024-01-31,90659,ASP INTEGRA OPC
2,NaN,019d425c-787f-4b39-99bd-e6641505ee39,NaN,NaN,NaN,ASC8GPZ5Q,2024-01-31,"1,500.61",PAID,"Opciones Empresariales del Noreste, S.A. de C....",...,'659802020000267294,2024-01-31T14:00:16Z,BANORTE,"Jose Guadalupe Herrera Acosta,,1500.61,6598020...",BNTE-3 2024-01-31.csv,659802020000267294,659,2024-01-31,90659,ASP INTEGRA OPC
3,NaN,023aeb3b-712b-4472-a8c6-17a42db36c6f,NaN,NaN,NaN,AMHFXR2F7,2024-01-31,179.19,PAID,"Opciones Empresariales del Noreste, S.A. de C....",...,'659802020000208334,2024-01-31T14:00:16Z,BANORTE,"Alejandro Ponce Estrada,,179.19,65980202000020...",BNTE-3 2024-01-31.csv,659802020000208334,659,2024-01-31,90659,ASP INTEGRA OPC
4,NaN,04bc1b25-2e2c-4389-a017-d6a39cc67204,NaN,NaN,NaN,A9TNVMKB3,2024-01-31,"1,002.80",PAID,"BBVA Bancomer, S.A.",...,'012760001859288342,2024-01-31T14:00:17Z,BANORTE,"Tersesita Rivas ,,1002.8,012760001859288342,CL...",BNTE-3 2024-01-31.csv,012760001859288342,012,2024-01-31,40012,BBVA BANCOMER


In [8]:
query_final.shape

(460, 22)

# PAGOS DUPLICADOS 

In [9]:
duplicates_query = query_final[query_final.duplicated(['report_id'],keep=False)]
duplicates_query.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,...,bank_account_number,batch_created_date,disbursement_method,swap,file_name,CLABE,BIN,date_disbursement,Clave de la institución,Nombre de la institución


In [10]:
duplicates_query.shape

(0, 22)

# VALIDACION MONTO A DISPERSAR

In [11]:
valid_0 = query_final
valid_0.loc[valid_0['Nombre de la institución'].str.contains('BANORTE/IXE'), 'comision_dsp'] = 1.05
valid_0.loc[~valid_0['Nombre de la institución'].str.contains('BANORTE/IXE'), 'comision_dsp'] = 1.80
valid_0['iva_dsp'] = valid_0['comision_dsp']*0.1600

In [12]:
valid_1 = valid_0.groupby(['date_disbursement']).agg({'amount':sum,'report_id':'count','comision_dsp':sum
                                                      ,'iva_dsp':sum}).reset_index()
valid_1.rename(columns={'report_id':'payments_number'},inplace=True)
valid_1['comision + iva'] = valid_1['comision_dsp'] + valid_1['iva_dsp']
valid_1['total_fondeo'] = valid_1['amount'] + valid_1['comision + iva']
valid_1['monto_mail'] = monto_me_mail
valid_1['comision_mail'] = comision_bnte_mail
valid_1['iva_mail'] = iva_bnte_mail
valid_1['monto_fondeo_mail'] = monto_fondear_mail
valid_1['check_monto'] = valid_1['amount'] - valid_1['monto_mail']
valid_1['check_comision'] = valid_1['comision_dsp'] - valid_1['comision_mail']
valid_1['check_iva'] = valid_1['iva_dsp'] - valid_1['iva_mail']
valid_1['check_fondeo'] = valid_1['total_fondeo'] - valid_1['monto_fondeo_mail']
valid_1['dsp'] = 'BNTE'
valid_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_8760\507955033.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  valid_1 = valid_0.groupby(['date_disbursement']).agg({'amount':sum,'report_id':'count','comision_dsp':sum


,date_disbursement,amount,payments_number,comision_dsp,iva_dsp,comision + iva,total_fondeo,monto_mail,comision_mail,iva_mail,monto_fondeo_mail,check_monto,check_comision,check_iva,check_fondeo,dsp
0,2024-01-31,"416,929.49",460,823.50,131.76,955.26,"417,884.75","416,929.49",823.50,131.76,"417,884.75",0.00,0.00,0.00,0.00,BNTE


In [13]:
valid_2 = valid_1[['date_disbursement','dsp','payments_number','amount','comision_dsp','iva_dsp','comision + iva','total_fondeo']]
valid_2.head()

,date_disbursement,dsp,payments_number,amount,comision_dsp,iva_dsp,comision + iva,total_fondeo
0,2024-01-31,BNTE,460,"416,929.49",823.50,131.76,955.26,"417,884.75"


In [14]:
valid_montos_final = valid_2.stack().reset_index(1)
valid_montos_final.rename(columns={'level_1':'concept',0:'data'},inplace=True)
valid_montos_final

,concept,data
0,date_disbursement,2024-01-31 00:00:00
0,dsp,BNTE
0,payments_number,460
0,amount,"416,929.49"
0,comision_dsp,823.50
0,iva_dsp,131.76
0,comision + iva,955.26
0,total_fondeo,"417,884.75"


# EXCEL

In [15]:
writer = pd.ExcelWriter(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\VALIDATION\VALIDATION-BNTE-3 2024-01-31.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
query.to_excel(writer, sheet_name='DATA QUERY',index=False)
valid_montos_final.to_excel(writer, sheet_name='MONTOS',index=False)
valid_1.to_excel(writer, sheet_name='DIFF MONTOS',index=False)
duplicates_query.to_excel(writer, sheet_name='PAGOS DUPLICADOS',index=False)
writer.close()

In [16]:
valid_1.to_csv(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\MONTOS EMAIL\DIFF MONTOS-BNTE-3 2024-01-31.csv',index=False)

In [17]:
valid_2.to_csv(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\VALIDATION\total_disbursement-BNTE-3 2024-01-31.csv',index=False)

In [18]:
duplicates_query.to_csv(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\VALIDATION\Duplicados-BNTE-3 2024-01-31.csv',index=False)